In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Loader

In [2]:
import torch
import numpy as np
import pandas as pd
def prepare_data_main_model(df, seq_length, output_size, date_range):
    '''
    df: pandas df contain all the data
    seq_length: number of days consider as input
    output_size: number of days to predict
    date_range: length of history to consider
    Output:
    - full_data: Prepared data as a list of tuples (input_sequence, target_labels).
    - state_ordered: List of unique states in the data.
    '''
    full_data = []
    state_ordered = []

    for state in df.index.get_level_values('fips').unique():
        df_state = df.iloc[df.index.get_level_values('fips') == state]
 # If the length of data is less than or equal to date_range, consider all data
        if len(df_state) <= date_range:
            L = len(df_state.to_numpy())
            train_state = []
            for i in range(L - seq_length - output_size + 1):
                train_seq = df_state.to_numpy()[i:i + seq_length]
                train_label = df_state.to_numpy()[i:i + seq_length + output_size][seq_length:seq_length + output_size,
                              0]
                train_state.append((train_seq, train_label))

            for x in train_state:
                full_data.append(x)
            state_ordered.append(state)
# If data is larger than date_range, consider the most recent date_range data
        else:
            df_state = df.iloc[df.index.get_level_values('fips') == state][-date_range:]

            train_state = []

            L = len(df_state.to_numpy())
            for i in range(L - seq_length - output_size + 1):
                train_seq = df_state.to_numpy()[i:i + seq_length]
                train_label = df_state.to_numpy()[i:i + seq_length + output_size][seq_length:seq_length + output_size,
                              0]
                train_state.append((train_seq, train_label))

            for x in train_state:
                full_data.append(x)
            state_ordered.append(state)
    return full_data, state_ordered

def splitdata(full_data, ratio, batch_size):
    '''
    Split the prepared data into training and testing datasets.

    Parameters:
    - full_data: Prepared data as a list of tuples (input_sequence, target_labels).
    - ratio: Ratio of data to be used for training (e.g., 0.8 for 80% training, 20% testing).
    - batch_size: Size of mini-batches for DataLoader.

    Output:
    - train_loader: DataLoader for the training dataset.
    - test_loader: DataLoader for the testing dataset.
    '''
    train_size = int(ratio * len(full_data))
    test_size = len(full_data) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(full_data, [train_size, test_size])

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True, drop_last=True)

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                              batch_size=batch_size,
                                              shuffle=True, drop_last=True)

    return train_loader, test_loader

#gd = pd.read_csv('/content/drive/MyDrive/Flu Forecasting/code/groundtruth.csv')
gd = pd.read_csv('/content/drive/MyDrive/Flu Forecasting/code/groundtruth2m.csv')
gd['Week_end']  = pd.to_datetime(gd['Week_end'])


In [3]:
region_fips = {
    1: ["09", "23", "25", "33", "44", "50"],  # Region 1 - Boston
    2: ["34", "36"],                          # Region 2 - New York
    3: ["10", "24", "42", "51", "54"],  # Region 3 - Philadelphia
    4: ["01", "12", "13", "21", "28", "37", "45", "47"],  # Region 4 - Atlanta
    5: ["17", "18", "26", "27", "39", "55"],  # Region 5 - Chicago
    6: ["05", "22", "35", "40", "48"],        # Region 6 - Dallas
    7: ["19", "20", "29", "31"],              # Region 7 - Kansas City
    8: ["08", "30", "38", "46", "49", "56"],  # Region 8 - Denver
    9: ["04", "06", "15", "32"],              # Region 9 - San Francisco
    10: ["02", "16", "41", "53"]              # Region 10 - Seattle
}

# Models

In [4]:
import torch
import pandas as pd
from torch import nn
import torch.nn.functional as F
pd.options.mode.chained_assignment = None

# LSTM

## Target: Rate ( rate = total admission * 100000/population) Current Version!!!

In [5]:


class LSTM(nn.Module):

    def __init__(self, input_size, hidden_layer_size, num_layers, output_size, dropout_rate):
        super().__init__()

        self.input_size = input_size

        self.hidden_layer_size = hidden_layer_size

        self.num_layers = num_layers

        self.output_size = output_size

        self.lstm = nn.LSTM(self.input_size, hidden_layer_size, num_layers, batch_first=True, dropout=dropout_rate)

        self.linear = nn.Linear(hidden_layer_size, 1000)

        self.dropout = nn.Dropout(dropout_rate)

        self.linear2 = nn.Linear(1000, output_size)

    def forward(self, input_seq):
        h = (torch.zeros(self.num_layers, input_seq.size(0), self.hidden_layer_size).to(device),
             torch.zeros(self.num_layers, input_seq.size(0), self.hidden_layer_size).to(device))

        lstm_out, self.hidden_cell = self.lstm(input_seq, h)

        # only return the results for last sequence
        lstm_out = lstm_out[:, -1, :]
        predictions = self.linear(lstm_out)
        #predictions = F.relu(predictions)
        #predictions = F.softplus(predictions)
        predictions = F.leaky_relu(predictions, negative_slope=0.01)
        predictions = self.dropout(predictions)
        predictions = self.linear2(predictions)

        return predictions

#### Main


In [6]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import trange
from datetime import date, timedelta

from collections import OrderedDict
from collections import namedtuple
from itertools import product

import torch.nn.functional as F


data_file = '/content/drive/MyDrive/Flu Forecasting/code/trainingdata_rate0117.csv'
model_file = '/content/drive/MyDrive/Flu Forecasting/LSTM/weightselection/region/'
prediction_file = '/content/drive/MyDrive/Flu Forecasting/LSTM/weightselection/region/'
summary_file ='/content/drive/MyDrive/Flu Forecasting/LSTM/weightselection/region/'
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


'''
params = OrderedDict(
    target = ['rate'], ### Our target variable
    lr = [0.00025],
    batch_size = [2],
    seq_length = [4,6,8,12], # length of input sequence to predict values
    region = [1,2,3,4,5,6,7,8,9,10], # specify the region
    output_size = [4], # we are predicting next four weeks admissions
    num_pred_features = [1],
    week_range = [64], # number of historical data (per state) used to train the model
    input_size = [15], # total number of input features
    hidden_layer_size = [64,128,256], #test
    num_layers = [1,2], #test
    ratio = [0.7],  # ratio of training set and validation set
    num_epochs = [50],
    dropout_rate = [0.8],
    lossfunc = [nn.SmoothL1Loss(beta=0.05, reduction = 'sum'),

                ]
)


class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs

pd.options.mode.chained_assignment = None

runs = RunBuilder.get_runs(params)
'''



# Constant parameters
constant_params = {
    'target': 'rate',  # and other parameters that you don't want to vary
    'lr': 0.00025,
    'batch_size': 2,
    'input_size': 15, # total number of features
    'output_size': 4, # number of weeks to predict
    'num_pred_features': 1,
    'week_range': 64, # number of historical data (per state) used to train
    'ratio': 0.7,
    'num_epochs':50,
    'dropout_rate': 0.8,
    'lossfunc': nn.SmoothL1Loss(beta=0.05, reduction = 'sum')


}

# Variable parameters and their specific combinations
variable_combinations = [
    {'seq_length': 6, 'hidden_layer_size': 256, 'num_layers': 1,'region':1},
    {'seq_length': 8, 'hidden_layer_size': 256, 'num_layers': 2,'region':2},
    {'seq_length': 6, 'hidden_layer_size': 128, 'num_layers': 1,'region':3},
    {'seq_length': 8, 'hidden_layer_size': 64, 'num_layers': 2,'region':4},
    {'seq_length': 8, 'hidden_layer_size': 128, 'num_layers': 2,'region':5},
    {'seq_length': 6, 'hidden_layer_size': 256, 'num_layers': 1,'region':6},
    {'seq_length': 8, 'hidden_layer_size': 128, 'num_layers': 1,'region':7},
    {'seq_length': 8, 'hidden_layer_size': 128, 'num_layers': 2,'region':8},
    {'seq_length': 8, 'hidden_layer_size': 128, 'num_layers': 1,'region':9},
    {'seq_length': 4, 'hidden_layer_size': 64, 'num_layers': 1,'region':10}
    # Add more combinations as needed
]

class RunBuilder():
    @staticmethod
    def get_runs(constant_params, variable_combinations):
        Run = namedtuple('Run', list(constant_params.keys()) + list(variable_combinations[0].keys()))

        runs = []
        for combo in variable_combinations:
            # Merge the constant and variable parameters
            params = {**constant_params, **combo}
            runs.append(Run(**params))

        return runs
# output dict
columns = [
    'Model',
    'region',
    'lr',
    'batch_size',
    'seq_length',
    'week_range',
    'hidden_layer_size',
    'num_layers',
    'lossfunc',
    'mse_validation',
    'mse_validation_1w',
    'mse_validation_2w',
    'mse_validation_3w',
    'mse_validation_4w',
    'Target'
]
# output dataframe
df_summary = pd.DataFrame(columns=columns)
runs = RunBuilder.get_runs(constant_params, variable_combinations)

pd.options.mode.chained_assignment = None



for run in RunBuilder.get_runs(constant_params, variable_combinations):
    if run.target == 'rate':
        results_file = 'rate'

    validation_predictions = []
    validation_labels = []
    # read data
    df = pd.read_csv(data_file)
    #df = df.rename(columns={'End Date': 'Week_end', 'State': 'fips'})
    df['Week_end'] = pd.to_datetime(df['Week_end'])
    # too many missing values in State'11'
    df =  df[df['fips'] != 11]
    #df =  df[df['fips'] == 1]
    #df = df[['fips', 'Week_end', 'total_admissions']]

# select region
    df['fips'] = df['fips'].astype(str).str.zfill(2)
    df = df[df['fips'].isin(region_fips[run.region])]
    '''
    # choose data in 2023-11 as test data  test 10-01
    start_date1 = pd.to_datetime('2023-11-01') - timedelta(weeks=run.seq_length )
    df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] < pd.to_datetime('2023-11-18'))] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2023-11 as training data and validation data
    df_train = df[(df['Week_end'] <= pd.to_datetime('2023-11-01'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set
    df_train = df_train.set_index(['fips', 'Week_end'])

    '''
    # test set is the week from 2024-01-06
    start_date1 = pd.to_datetime('2024-01-06') - timedelta(weeks=run.seq_length)
    df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] <pd.to_datetime('2024-01-13'))] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2024-01-06 as training data and validation data and after 2022-08-01
    df_train = df[(df['Week_end'] <= pd.to_datetime('2023-12-30'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set
    #df_train = df[(df['Week_end'] <= pd.to_datetime('2024-01-06'))& (df['Week_end'] >=pd.to_datetime('2023-01-01')) ]
    df_train = df_train.set_index(['fips', 'Week_end'])

    #rescale the data from 0 to 1, normalization
    first_col = df_train.pop(run.target)
    df_train.insert(0, run.target, first_col)
    first_col_2 = df_test.pop(run.target)
    df_test.insert(0, run.target, first_col_2)

    scaler = MinMaxScaler(feature_range=(0, 1))

    scaler.fit(df_train.iloc[:, 1:])
    train_features_normalized = scaler.transform(df_train.iloc[:, 1:])
    test_feature_normalized = scaler.transform(df_test.iloc[:, 1:])

    scaler_target = MinMaxScaler(feature_range=(0, 1))
    scaler_target.fit(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    train_target_normalized = scaler_target.transform(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    test_target_normalized = scaler_target.transform(np.asarray(df_test.iloc[:, 0]).reshape(-1, 1))

    df_train.iloc[:, 1:] = train_features_normalized
    df_train.iloc[:, 0] = train_target_normalized
    df_test.iloc[:, 1:] = test_feature_normalized
    df_test.iloc[:, 0] = test_target_normalized

    '''
    Training
    '''
    # prepare the training and validation data (sequance format)
    full_data_main, state_ordered = prepare_data_main_model(df_train, run.seq_length,
                                                            run.output_size, run.week_range)

    model_main = LSTM(run.input_size, run.hidden_layer_size, run.num_layers, run.output_size ,
                      run.dropout_rate).to(device)
    # split the training data into training set and validation set
    train_loader_main, test_loader_main = splitdata(full_data_main, run.ratio, run.batch_size)

    loss_function = run.lossfunc
  # add l2 regularization
    optimizer_main = torch.optim.Adam(model_main.parameters(), lr=run.lr,weight_decay=1e-5)

    track_loss_train = []
    track_loss_test = []
    best_loss = 100000
    ###

    best_mse_numerical = None
    mse_numerical_weekly = [None] * 4

    best_mse_numerical_weekly = [None] * 4
    for i in trange(run.num_epochs):
# Set the model in training mode
        model_main.train()
        epoch_loss_train = 0
# Iterate over the training data
        for i, (seq, labels) in enumerate(train_loader_main):
            seq, labels = seq.to(device), labels.to(device)
            optimizer_main.zero_grad()
            seq = torch.as_tensor(seq).reshape(-1, run.seq_length, run.input_size)
             # Initialize the hidden state of the LSTM
            model_main.hidden_cell = (torch.zeros(run.num_layers, seq.size()[0], run.hidden_layer_size).to(device),
                          torch.zeros(run.num_layers, seq.size()[0], run.hidden_layer_size).to(device))

            # Forward pass to get predictions
            y_pred = model_main(seq.float())

            # Calculate the loss and perform backpropagation
            single_loss = loss_function(y_pred, torch.as_tensor(labels).float())
            single_loss.backward()
            optimizer_main.step()

            epoch_loss_train += single_loss.item()
        # Track the training loss for this epoch
        track_loss_train.append(epoch_loss_train)

        with torch.no_grad():
          # Set the model in evaluation mode
            model_main.eval()
            epoch_loss_test = 0
            ########
            # Initialize lists to store validation predictions and labels
            validation_predictions = []
            validation_labels = []
            for i, (seq, labels) in enumerate(test_loader_main):
                seq, labels = seq.to(device), labels.to(device)
                seq = torch.as_tensor(seq).reshape(-1, run.seq_length, run.input_size)
                model_main.hidden_cell = (torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device),
                                 torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device))
                y_pred = model_main(seq.float())

                single_loss = loss_function(y_pred, torch.as_tensor(labels).float())
                epoch_loss_test += single_loss.item()
                ######
                # Store predictions and labels for later evaluation
                validation_predictions.extend(y_pred.cpu().numpy())
                validation_labels.extend(labels.cpu().numpy())
            track_loss_test.append(epoch_loss_test)
        # Combine validation predictions and labels, and perform inverse scaling to get true value
        validation_predictions = np.concatenate(validation_predictions, axis=0)
        validation_labels = np.concatenate(validation_labels, axis=0)
        validation_predictions = scaler_target.inverse_transform(validation_predictions.reshape(-1, 1)).reshape(-1, 4)
        validation_labels = scaler_target.inverse_transform(validation_labels.reshape(-1, 1)).reshape(-1, 4)

        # Calculate total and weekly MSE and accuracy for validation set
        mse_numerical = np.mean((validation_predictions - validation_labels)**2)

        for week in range(4):
            # Calculate weekly MSE
            mse_numerical_week = np.mean((validation_predictions[:, week] - validation_labels[:, week])**2)
            mse_numerical_weekly[week] = mse_numerical_week


        if epoch_loss_test  < best_loss:
          # Update the best loss and save the model; perform early stopping
            best_mse_numerical = mse_numerical
            best_mse_numerical_weekly = mse_numerical_weekly

            best_loss = epoch_loss_test
            print('Train Loss: ', epoch_loss_train)
            print('Test Loss: ', epoch_loss_test)
            es = 0
            torch.save(model_main.state_dict(),
                           model_file + run.target +'_' +str(run.region)+ '_' +
                           str(run.week_range) + '_' + str(run.hidden_layer_size)\
                           + '_' + str(run.seq_length) + '_' + str(run.num_layers)\
                           + '_LSTM_weights.pt')
        else:
            es += 1
            print("Counter {} of 5".format(es))
            print('Train Loss: ', epoch_loss_train)
            print('Test Loss: ', epoch_loss_test)


        if es > 4:
            print("Early stopping with best_loss: ", best_loss, "and test_loss for this epoch: ",
                      epoch_loss_test,
                      "...")

            break

    '''
    Prediction
    '''
    df_output = pd.DataFrame(columns=['fips', 'Week_end', 'Prediction_1w',
                                      'Prediction_2w', 'Prediction_3w',
                                      'Prediction_4w'])
  #test set starts with 2023-11-01
    test_weeks = df_test[df_test.index.get_level_values('Week_end') \
                       >= pd.to_datetime('2024-01-06')].index.get_level_values('Week_end').unique()
    test_states = df_test.index.get_level_values('fips').unique()

    m_state_dict_main = torch.load(model_file + run.target+ '_' +str(run.region)+ '_' +
                                   str(run.week_range) + '_' + str(run.hidden_layer_size) \
                                   + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                                   + '_LSTM_weights.pt')

    model_main = LSTM(run.input_size, run.hidden_layer_size, run.num_layers,
                      run.output_size, run.dropout_rate).to(device)
    model_main.load_state_dict(m_state_dict_main)

    with torch.no_grad():
        #model_main.eval()
        for fips in test_states:
            for week in test_weeks:
                seq = df_test[(df_test.index.get_level_values('fips') == fips)\
                 & (df_test.index.get_level_values('Week_end') <= week)][-run.seq_length:].to_numpy()

                seq = torch.tensor(seq).reshape(-1, run.seq_length, run.input_size).to(device)


                model_main.hidden_cell = (torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device),
                                 torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device))
                prediction = model_main(seq.float())

                prediction = scaler_target.inverse_transform(prediction.cpu().detach().numpy().reshape(-1, 1))
                # save prediction results
                dic = {
                    'fips' : fips,
                    'Week_end' : week,
                    'Prediction_1w' : prediction[0].item(),
                    'Prediction_2w' : prediction[1].item(),
                    'Prediction_3w' : prediction[2].item(),
                    'Prediction_4w' : prediction[3].item()

                }

                df_output = pd.concat([df_output, pd.DataFrame([dic])], ignore_index=True)


        df_output.to_csv(prediction_file + run.target + '_' +str(run.region)+ '_' +
                            str(run.week_range) + '_' + str(run.hidden_layer_size) \
                            + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                            + '.csv')

        '''
        Evaluation
        '''





        dic_lstm = {
            'Model': 'LSTM',
            'region': run.region,
            'lr': run.lr,
            'batch_size': run.batch_size,
            'seq_length': run.seq_length,
            'week_range': run.week_range,
            'hidden_layer_size': run.hidden_layer_size,
            'num_layers': run.num_layers,
            'lossfunc': run.lossfunc,

            'mse_validation': best_mse_numerical,

            'mse_validation_1w': best_mse_numerical_weekly[0],
            'mse_validation_2w': best_mse_numerical_weekly[1],
            'mse_validation_3w': best_mse_numerical_weekly[2],
            'mse_validation_4w': best_mse_numerical_weekly[3],

            'Target': run.target
        }

        print('mse_validation:')
        print(dic_lstm['mse_validation'])


        df_summary = pd.concat([df_summary, pd.DataFrame([dic_lstm])], ignore_index=True)

        df_summary.to_csv(summary_file + 'summarytest.csv')



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
  0%|          | 0/50 [00:00<?, ?it/s]

Train Loss:  51.011956482194364
Test Loss:  18.11551872268319


  4%|▍         | 2/50 [00:03<01:06,  1.38s/it]

Train Loss:  45.00931313633919
Test Loss:  16.537852904759347


  6%|▌         | 3/50 [00:03<00:43,  1.08it/s]

...

Counter 4 of 5
Train Loss:  27.086908216122538
Test Loss:  17.963626876007766


 24%|██▍       | 12/50 [00:03<00:12,  3.15it/s]

Counter 5 of 5
Train Loss:  25.288568222429603
Test Loss:  17.048198079224676
Early stopping with best_loss:  17.022779367864132 and test_loss for this epoch:  17.048198079224676 ...



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


mse_validation:
8.704891548097683


## Run best model for 100 times

In [7]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import trange
from datetime import date, timedelta

from collections import OrderedDict
from collections import namedtuple
from itertools import product

import torch.nn.functional as F


n_result = pd.DataFrame()
data_file = '/content/drive/MyDrive/Flu Forecasting/code/trainingdata_rate0117.csv'
model_file = '/content/drive/MyDrive/Flu Forecasting/LSTM/weightselection/region/'
prediction_file = '/content/drive/MyDrive/Flu Forecasting/LSTM/weightselection/region/'
summary_file ='/content/drive/MyDrive/Flu Forecasting/LSTM/weightselection/region/'
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# Constant parameters
constant_params = {
    'target': 'rate',  # and other parameters that you don't want to vary
    'lr': 0.00025,
    'batch_size': 2,
    'input_size': 15, # total number of features
    'output_size': 4, # number of weeks to predict
    'num_pred_features': 1,
    'week_range': 64, # number of historical data (per state) used to train
    'ratio': 0.7,
    'num_epochs':50,
    'dropout_rate': 0.8,
    'lossfunc': nn.SmoothL1Loss(beta=0.05, reduction = 'sum')


}

# Variable parameters and their specific combinations
variable_combinations = [
    {'seq_length': 6, 'hidden_layer_size': 256, 'num_layers': 1,'region':1},
    {'seq_length': 8, 'hidden_layer_size': 256, 'num_layers': 2,'region':2},
    {'seq_length': 6, 'hidden_layer_size': 128, 'num_layers': 1,'region':3},
    {'seq_length': 8, 'hidden_layer_size': 64, 'num_layers': 2,'region':4},
    {'seq_length': 8, 'hidden_layer_size': 128, 'num_layers': 2,'region':5},
    {'seq_length': 6, 'hidden_layer_size': 256, 'num_layers': 1,'region':6},
    {'seq_length': 8, 'hidden_layer_size': 128, 'num_layers': 1,'region':7},
    {'seq_length': 8, 'hidden_layer_size': 128, 'num_layers': 2,'region':8},
    {'seq_length': 8, 'hidden_layer_size': 128, 'num_layers': 1,'region':9},
    {'seq_length': 4, 'hidden_layer_size': 64, 'num_layers': 1,'region':10}
    # Add more combinations as needed
]

class RunBuilder():
    @staticmethod
    def get_runs(constant_params, variable_combinations):
        Run = namedtuple('Run', list(constant_params.keys()) + list(variable_combinations[0].keys()))

        runs = []
        for combo in variable_combinations:
            # Merge the constant and variable parameters
            params = {**constant_params, **combo}
            runs.append(Run(**params))

        return runs

# Generate runs with selected combinations
runs = RunBuilder.get_runs(constant_params, variable_combinations)

pd.options.mode.chained_assignment = None



for run in RunBuilder.get_runs(constant_params, variable_combinations):
    if run.target == 'rate':
        results_file = 'rate'

    validation_predictions = []
    validation_labels = []
    # read data
    df = pd.read_csv(data_file)
    #df = df.rename(columns={'End Date': 'Week_end', 'State': 'fips'})
    df['Week_end'] = pd.to_datetime(df['Week_end'])
    # too many missing values in State'11'
    df =  df[df['fips'] != 11]
    #df =  df[df['fips'] == 1]
    #df = df[['fips', 'Week_end', 'total_admissions']]
    #df = df.drop(['PRCP_mean', 'SNOW_mean', 'TMAXDELTA', 'TMINDELTA'], axis=1)
    # choose data in 2023-11 as test data  test 10-01


    df['fips'] = df['fips'].astype(str).str.zfill(2)
    df = df[df['fips'].isin(region_fips[run.region])]
    '''
    # choose data in 2023-11 as test data  test 10-01
    start_date1 = pd.to_datetime('2023-11-01') - timedelta(weeks=run.seq_length )
    df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] < pd.to_datetime('2023-11-18'))] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2023-11 as training data and validation data
    df_train = df[(df['Week_end'] <= pd.to_datetime('2023-11-01'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set
    df_train = df_train.set_index(['fips', 'Week_end'])

    #rescale the data from 0 to 1, normalization
    first_col = df_train.pop(run.target)
    df_train.insert(0, run.target, first_col)
    first_col_2 = df_test.pop(run.target)
    df_test.insert(0, run.target, first_col_2)
    '''
    start_date1 = pd.to_datetime('2024-01-06') - timedelta(weeks=run.seq_length)
    df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] <pd.to_datetime('2024-01-13'))] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2024-01-06 as training data and validation data and after 2022-08-01
    df_train = df[(df['Week_end'] <= pd.to_datetime('2023-12-30'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set
    #df_train = df[(df['Week_end'] <= pd.to_datetime('2024-01-06'))& (df['Week_end'] >=pd.to_datetime('2023-01-01')) ]
    df_train = df_train.set_index(['fips', 'Week_end'])

    #rescale the data from 0 to 1, normalization
    first_col = df_train.pop(run.target)
    df_train.insert(0, run.target, first_col)
    first_col_2 = df_test.pop(run.target)
    df_test.insert(0, run.target, first_col_2)

    scaler = MinMaxScaler(feature_range=(0, 1))

    scaler.fit(df_train.iloc[:, 1:])
    train_features_normalized = scaler.transform(df_train.iloc[:, 1:])
    test_feature_normalized = scaler.transform(df_test.iloc[:, 1:])

    scaler_target = MinMaxScaler(feature_range=(0, 1))
    scaler_target.fit(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    train_target_normalized = scaler_target.transform(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    test_target_normalized = scaler_target.transform(np.asarray(df_test.iloc[:, 0]).reshape(-1, 1))

    df_train.iloc[:, 1:] = train_features_normalized
    df_train.iloc[:, 0] = train_target_normalized
    df_test.iloc[:, 1:] = test_feature_normalized
    df_test.iloc[:, 0] = test_target_normalized


    for i in range(1000):
      df_output = pd.DataFrame(columns=['fips', 'Week_end', 'Prediction_1w',
                                        'Prediction_2w', 'Prediction_3w',
                                        'Prediction_4w'])
    #test set starts with 2024-01-06
      test_weeks = df_test[df_test.index.get_level_values('Week_end') \
                          >= pd.to_datetime('2024-01-06')].index.get_level_values('Week_end').unique()
      test_states = df_test.index.get_level_values('fips').unique()

      m_state_dict_main = torch.load(model_file + run.target + '_' +str(run.region)+ '_' +
                                      str(run.week_range) + '_' + str(run.hidden_layer_size) \
                                      + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                                      + '_LSTM_weights.pt')

      model_main = LSTM(run.input_size, run.hidden_layer_size, run.num_layers,
                        run.output_size, run.dropout_rate).to(device)
      model_main.load_state_dict(m_state_dict_main)

      with torch.no_grad():
          #model_main.eval()
          for fips in test_states:
              for week in test_weeks:
                  seq = df_test[(df_test.index.get_level_values('fips') == fips)\
                    & (df_test.index.get_level_values('Week_end') <= week)][-run.seq_length:].to_numpy()

                  seq = torch.tensor(seq).reshape(-1, run.seq_length, run.input_size).to(device)


                  model_main.hidden_cell = (torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device),
                                    torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device))
                  prediction = model_main(seq.float())

                  prediction = scaler_target.inverse_transform(prediction.cpu().detach().numpy().reshape(-1, 1))
                  # save prediction results
                  dic = {
                      'fips' : fips,
                      'Week_end' : week,
                      'Prediction_1w' : prediction[0].item(),
                      'Prediction_2w' : prediction[1].item(),
                      'Prediction_3w' : prediction[2].item(),
                      'Prediction_4w' : prediction[3].item()

                  }

                  df_output = pd.concat([df_output, pd.DataFrame([dic])], ignore_index=True)


          df_output.to_csv(prediction_file + run.target + '_' +str(run.region)+ '_' +
                              str(run.week_range) + '_' + str(run.hidden_layer_size) \
                              + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                              + '.csv')
          n_result= pd.concat([n_result, df_output], ignore_index=True)


    n_result.to_pickle(summary_file + 'allresulttest0110.pkl')

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/

In [8]:
n_result = n_result.sort_values(by='fips')
n_result
#n_result.to_pickle(summary_file + 'allresulttest1223.pkl')

,fips,Week_end,Prediction_1w,Prediction_2w,Prediction_3w,Prediction_4w
16728,01,2024-01-06,3.820290,2.822184,2.659551,3.181714
16392,01,2024-01-06,2.990170,3.403265,4.016337,4.541819
15280,01,2024-01-06,4.757852,5.347331,3.922728,4.050310
16384,01,2024-01-06,4.374333,3.424502,3.266654,3.204413
13800,01,2024-01-06,4.597184,4.257525,4.798559,5.155047
...,...,...,...,...,...,...
38831,56,2024-01-06,6.123800,5.291426,5.342564,3.927052
38825,56,2024-01-06,8.028559,6.571862,5.389025,3.572135
38819,56,2024-01-06,3.587781,2.850131,2.990027,2.401264
38945,56,2024-01-06,10.601819,7.979209,7.237325,4.866913


# GRU

In [ ]:
class GRU(nn.Module):

    def __init__(self, input_size, hidden_layer_size, num_layers, output_size, dropout_rate):
        super().__init__()

        self.input_size = input_size
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers
        self.output_size = output_size


        self.gru = nn.GRU(self.input_size, hidden_layer_size, num_layers, batch_first=True, dropout=dropout_rate)

        self.linear = nn.Linear(hidden_layer_size, 1000)
        self.dropout = nn.Dropout(dropout_rate)
        self.linear2 = nn.Linear(1000, output_size)

    def forward(self, input_seq):
        h = torch.zeros(self.num_layers, input_seq.size(0), self.hidden_layer_size).to(device)


        gru_out, self.hidden = self.gru(input_seq, h)


        gru_out = gru_out[:, -1, :]
        predictions = self.linear(gru_out)
        predictions = F.relu(predictions)
        predictions = self.dropout(predictions)
        predictions = self.linear2(predictions)

        return predictions


### Main

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import trange
from datetime import date, timedelta

from collections import OrderedDict
from collections import namedtuple
from itertools import product

import torch.nn.functional as F


'''
Don't save trained model or results in github, there is no enough space
'''
data_file = '/content/drive/MyDrive/Flu Forecasting/code/trainingdata_rate0117.csv'
model_file = '/content/drive/MyDrive/Flu Forecasting/GRU/model for regions/'
prediction_file = '/content/drive/MyDrive/Flu Forecasting/GRU/result for regions/'
summary_file ='/content/drive/MyDrive/Flu Forecasting/GRU/result for regions/'
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


'''
params = OrderedDict(
    target = ['rate'], ### Our target variable
    lr = [0.00025],
    batch_size = [2],
    seq_length = [4,6,8,12], # length of input sequence to predict values
    region = [1,2,3,4,5,6,7,8,9,10], # specify the region
    output_size = [4], # we are predicting next four weeks admissions
    num_pred_features = [1],
    week_range = [64], # number of historical data (per state) used to train the model
    input_size = [15], # total number of input features
    hidden_layer_size = [64,128,256], #test
    num_layers = [1,2], #test
    ratio = [0.7],  # ratio of training set and validation set
    num_epochs = [50],
    dropout_rate = [0.8],
    lossfunc = [nn.SmoothL1Loss(beta=0.05, reduction = 'sum'),

                ]
)


class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs

pd.options.mode.chained_assignment = None

runs = RunBuilder.get_runs(params)
'''
# Constant parameters
constant_params = {
    'target': 'rate',  # and other parameters that you don't want to vary
    'lr': 0.00025,
    'batch_size': 2,
    'input_size': 15, # total number of features
    'output_size': 4, # number of weeks to predict
    'num_pred_features': 1,
    'week_range': 64, # number of historical data (per state) used to train
    'ratio': 0.7,
    'num_epochs':50,
    'dropout_rate': 0.8,
    'lossfunc': [nn.SmoothL1Loss(beta=0.05, reduction = 'sum')]


}

# Variable parameters and their specific combinations
variable_combinations = [
    {'seq_length': 6, 'hidden_layer_size': 64, 'num_layers': 1,'region':1},
    {'seq_length': 8, 'hidden_layer_size': 128, 'num_layers': 2,'region':2},
    {'seq_length': 4, 'hidden_layer_size': 256, 'num_layers': 1,'region':3},
    {'seq_length': 12, 'hidden_layer_size': 128, 'num_layers': 2,'region':4},
    {'seq_length': 8, 'hidden_layer_size': 64, 'num_layers': 2,'region':5},
    {'seq_length': 6, 'hidden_layer_size': 128, 'num_layers': 2,'region':6},
    {'seq_length': 4, 'hidden_layer_size': 128, 'num_layers': 1,'region':7},
    {'seq_length': 8, 'hidden_layer_size': 128, 'num_layers': 2,'region':8},
    {'seq_length': 6, 'hidden_layer_size': 256, 'num_layers': 1,'region':9},
    {'seq_length': 12, 'hidden_layer_size': 128, 'num_layers': 1,'region':10}
    # Add more combinations as needed
]

class RunBuilder():
    @staticmethod
    def get_runs(constant_params, variable_combinations):
        Run = namedtuple('Run', list(constant_params.keys()) + list(variable_combinations[0].keys()))

        runs = []
        for combo in variable_combinations:
            # Merge the constant and variable parameters
            params = {**constant_params, **combo}
            runs.append(Run(**params))

        return runs

# Generate runs with selected combinations
runs = RunBuilder.get_runs(constant_params, variable_combinations)

# output dict
columns = [
    'Model',
    'region',
    'lr',
    'batch_size',
    'seq_length',
    'week_range',
    'hidden_layer_size',
    'num_layers',
    'lossfunc',
    'mse_validation',
    'mse_validation_1w',
    'mse_validation_2w',
    'mse_validation_3w',
    'mse_validation_4w',
    'Target'
]
# output dataframe
df_summary = pd.DataFrame(columns=columns)
for run in RunBuilder.get_runs(constant_params, variable_combinations):
    if run.target == 'rate':
        results_file = 'rate'

    validation_predictions = []
    validation_labels = []
    # read data
    df = pd.read_csv(data_file)
    #df = df.rename(columns={'End Date': 'Week_end', 'State': 'fips'})
    df['Week_end'] = pd.to_datetime(df['Week_end'])
    # too many missing values in State'11'
    df =  df[df['fips'] != 11]
    #df =  df[df['fips'] == 1]
    #df = df[['fips', 'Week_end', 'total_admissions']]

# select region
    df['fips'] = df['fips'].astype(str).str.zfill(2)
    df = df[df['fips'].isin(region_fips[run.region])]
    '''
    # choose data in 2023-11 as test data  test 10-01
    start_date1 = pd.to_datetime('2023-11-01') - timedelta(weeks=run.seq_length )
    df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] < pd.to_datetime('2023-11-18'))] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2023-11 as training data and validation data
    df_train = df[(df['Week_end'] <= pd.to_datetime('2023-11-01'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set
    df_train = df_train.set_index(['fips', 'Week_end'])

    '''
    # test set is the week from 2024-01-06
    start_date1 = pd.to_datetime('2024-01-13') - timedelta(weeks=run.seq_length )
    df_test = df[df['Week_end'] > start_date1] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2024-01-06 as training data and validation data and after 2022-08-01
    df_train = df[(df['Week_end'] <= pd.to_datetime('2024-01-06'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set
    #df_train = df[(df['Week_end'] <= pd.to_datetime('2024-01-06'))& (df['Week_end'] >=pd.to_datetime('2023-01-01')) ]
    df_train = df_train.set_index(['fips', 'Week_end'])

    #rescale the data from 0 to 1, normalization

    first_col = df_train.pop(run.target)
    df_train.insert(0, run.target, first_col)
    first_col_2 = df_test.pop(run.target)
    df_test.insert(0, run.target, first_col_2)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(df_train.iloc[:, 1:])
    train_features_normalized = scaler.transform(df_train.iloc[:, 1:])
    test_feature_normalized = scaler.transform(df_test.iloc[:, 1:])

    scaler_target = MinMaxScaler(feature_range=(0, 1))
    scaler_target.fit(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    train_target_normalized = scaler_target.transform(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    test_target_normalized = scaler_target.transform(np.asarray(df_test.iloc[:, 0]).reshape(-1, 1))

    df_train.iloc[:, 1:] = train_features_normalized
    df_train.iloc[:, 0] = train_target_normalized
    df_test.iloc[:, 1:] = test_feature_normalized
    df_test.iloc[:, 0] = test_target_normalized

    '''
    Training
    '''

    full_data_main, state_ordered = prepare_data_main_model(df_train, run.seq_length,
                                                            run.output_size, run.week_range)


    model_main = GRU(run.input_size, run.hidden_layer_size, run.num_layers, run.output_size, run.dropout_rate).to(device)


    train_loader_main, test_loader_main = splitdata(full_data_main, run.ratio, run.batch_size)

    loss_function = run.lossfunc

    optimizer_main = torch.optim.Adam(model_main.parameters(), lr=run.lr)

    track_loss_train = []
    track_loss_test = []
    best_loss = 100000
    ###
    best_mse_categorical = None
    best_accuracy_categorical = None
    best_mse_numerical = None
    mse_numerical_weekly = [None] * 4
    accuracy_categorical_weekly = [None] * 4
    mse_categorical_weekly = [None] * 4
    best_mse_categorical_weekly = [None] * 4
    best_accuracy_categorical_weekly = [None] * 4
    best_mse_numerical_weekly = [None] * 4
    for i in trange(run.num_epochs):

        model_main.train()
        epoch_loss_train = 0

        for i, (seq, labels) in enumerate(train_loader_main):
            seq, labels = seq.to(device), labels.to(device)
            optimizer_main.zero_grad()
            seq = torch.as_tensor(seq).reshape(-1, run.seq_length, run.input_size)
            model_main.hidden = torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device)

            y_pred = model_main(seq.float())

            single_loss = loss_function(y_pred, torch.as_tensor(labels).float())
            single_loss.backward()
            optimizer_main.step()

            epoch_loss_train += single_loss.item()

        track_loss_train.append(epoch_loss_train)
        model_main.eval()
        with torch.no_grad():
            epoch_loss_test = 0
          ########
            validation_predictions = []
            validation_labels = []
            for i, (seq, labels) in enumerate(test_loader_main):
                seq, labels = seq.to(device), labels.to(device)
                seq = torch.as_tensor(seq).reshape(-1, run.seq_length, run.input_size)
                model_main.hidden = torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device)
                y_pred = model_main(seq.float())

                single_loss = loss_function(y_pred, torch.as_tensor(labels).float())
                epoch_loss_test += single_loss.item()

                            ######
                validation_predictions.extend(y_pred.cpu().numpy())
                validation_labels.extend(labels.cpu().numpy())
            track_loss_test.append(epoch_loss_test)

        validation_predictions = np.concatenate(validation_predictions, axis=0)
        validation_labels = np.concatenate(validation_labels, axis=0)
        validation_predictions = scaler_target.inverse_transform(validation_predictions.reshape(-1, 1)).reshape(-1, 4)
        validation_labels = scaler_target.inverse_transform(validation_labels.reshape(-1, 1)).reshape(-1, 4)

        # Calculate total and weekly MSE and accuracy
        mse_numerical = np.mean((validation_predictions - validation_labels)**2)

        for week in range(4):
            # Numerical MSE
            mse_numerical_week = np.mean((validation_predictions[:, week] - validation_labels[:, week])**2)
            mse_numerical_weekly[week] = mse_numerical_week


        if epoch_loss_test  < best_loss:
          #####
            best_mse_numerical = mse_numerical
            best_mse_numerical_weekly = mse_numerical_weekly

            best_loss = epoch_loss_test
            print('Train Loss: ', epoch_loss_train)
            print('Test Loss: ', epoch_loss_test)
            es = 0
            torch.save(model_main.state_dict(),
                           model_file + run.target + '_' + str(run.region)+'_'+
                           str(run.week_range) + '_' + str(run.hidden_layer_size)\
                           + '_' + str(run.seq_length) + '_' + str(run.num_layers)\
                                                  + '_GRU_weights.pt')
        else:
            es += 1
            print("Counter {} of 5".format(es))
            print('Train Loss: ', epoch_loss_train)
            print('Test Loss: ', epoch_loss_test)


        if es > 4:
            print("Early stopping with best_loss: ", best_loss, "and test_loss for this epoch: ",
                      epoch_loss_test,
                      "...")

            break

    '''
    Prediction
    '''
    df_output = pd.DataFrame(columns=['fips', 'Week_end', 'Prediction_1w',
                                      'Prediction_2w', 'Prediction_3w',
                                      'Prediction_4w'])

    test_weeks = df_test[df_test.index.get_level_values('Week_end') \
                       >= pd.to_datetime('2024-01-13')].index.get_level_values('Week_end').unique()
    test_states = df_test.index.get_level_values('fips').unique()

    m_state_dict_main = torch.load(model_file + run.target + '_' + str(run.region)+'_'+
                                   str(run.week_range) + '_' + str(run.hidden_layer_size) \
                                   + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                                   + '_GRU_weights.pt')

    model_main = GRU(run.input_size, run.hidden_layer_size, run.num_layers, run.output_size, run.dropout_rate).to(device)

    model_main.load_state_dict(m_state_dict_main)

    with torch.no_grad():

        for fips in test_states:
            for week in test_weeks:
                seq = df_test[(df_test.index.get_level_values('fips') == fips)\
                 & (df_test.index.get_level_values('Week_end') <= week)][-run.seq_length:].to_numpy()

                seq = torch.tensor(seq).reshape(-1, run.seq_length, run.input_size).to(device)

                model_main.hidden = torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device)
                prediction = model_main(seq.float())

                prediction = scaler_target.inverse_transform(prediction.cpu().detach().numpy().reshape(-1, 1))

                dic = {
                    'fips' : fips,
                    'Week_end' : week,
                    'Prediction_1w' : prediction[0].item(),
                    'Prediction_2w' : prediction[1].item(),
                    'Prediction_3w' : prediction[2].item(),
                    'Prediction_4w' : prediction[3].item()

                }

                df_output = pd.concat([df_output, pd.DataFrame([dic])], ignore_index=True)


        df_output.to_csv(prediction_file + run.target + '_' + str(run.region)+'_'+
                            str(run.week_range) + '_' + str(run.hidden_layer_size) \
                            + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                            + '.csv')

        '''
        Evaluation
        '''



        dic_lstm = {
            'Model': 'GRU',
            'region': run.region,
            'lr': run.lr,
            'batch_size': run.batch_size,
            'seq_length': run.seq_length,
            'week_range': run.week_range,
            'hidden_layer_size': run.hidden_layer_size,
            'num_layers': run.num_layers,
            'lossfunc': run.lossfunc,

            'mse_validation': best_mse_numerical,

            'mse_validation_1w': best_mse_numerical_weekly[0],
            'mse_validation_2w': best_mse_numerical_weekly[1],
            'mse_validation_3w': best_mse_numerical_weekly[2],
            'mse_validation_4w': best_mse_numerical_weekly[3],

            'Target': run.target
        }

        print('mse_validation:')
        print(dic_lstm['mse_validation'])


        df_summary = pd.concat([df_summary, pd.DataFrame([dic_lstm])], ignore_index=True)

        df_summary.to_csv(summary_file + 'summarytest.csv')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
  0%|          | 0/50 [00:00<?, ?it/s]

Train Loss:  70.92459118366241
Test Loss:  21.077140668639913


  4%|▍         | 2/50 [00:02<00:54,  1.14s/it]

Train Loss:  52.87881154567003
Test Loss:  19.941471042111516


  6%|▌         | 3/50 [00:03<00:37,  1.24it/s]

...

Counter 3 of 5
Train Loss:  20.02266515418887
Test Loss:  6.939123585005291


 54%|█████▍    | 27/50 [00:07<00:06,  3.78it/s]

Counter 4 of 5
Train Loss:  14.804330930579454
Test Loss:  7.337002018932253


 54%|█████▍    | 27/50 [00:07<00:06,  3.57it/s]

Counter 5 of 5
Train Loss:  19.006741560529917
Test Loss:  7.265435907524079
Early stopping with best_loss:  5.656708731548861 and test_loss for this epoch:  7.265435907524079 ...
mse_validation:
1.5119260745901002


### run best model for 100 times

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import trange
from datetime import date, timedelta

from collections import OrderedDict
from collections import namedtuple
from itertools import product

import torch.nn.functional as F


n_result = pd.DataFrame()
data_file = '/content/drive/MyDrive/Flu Forecasting/code/trainingdata_rate0110.csv'
model_file = '/content/drive/MyDrive/Flu Forecasting/GRU/model for regions/'
prediction_file = '/content/drive/MyDrive/Flu Forecasting/GRU/result for regions/'
summary_file ='/content/drive/MyDrive/Flu Forecasting/GRU/result for regions/'
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




# Constant parameters
constant_params = {
    'target': 'rate',  # and other parameters that you don't want to vary
    'lr': 0.00025,
    'batch_size': 2,
    'input_size': 15, # total number of features
    'output_size': 4, # number of weeks to predict
    'num_pred_features': 1,
    'week_range': 64, # number of historical data (per state) used to train
    'ratio': 0.7,
    'num_epochs':50,
    'dropout_rate': 0.8,
    'lossfunc': [nn.SmoothL1Loss(beta=0.05, reduction = 'sum')]


}

# Variable parameters and their specific combinations
variable_combinations = [
    {'seq_length': 6, 'hidden_layer_size': 64, 'num_layers': 1,'region':1},
    {'seq_length': 8, 'hidden_layer_size': 128, 'num_layers': 2,'region':2},
    {'seq_length': 4, 'hidden_layer_size': 256, 'num_layers': 1,'region':3},
    {'seq_length': 12, 'hidden_layer_size': 128, 'num_layers': 2,'region':4},
    {'seq_length': 8, 'hidden_layer_size': 64, 'num_layers': 2,'region':5},
    {'seq_length': 6, 'hidden_layer_size': 128, 'num_layers': 2,'region':6},
    {'seq_length': 4, 'hidden_layer_size': 128, 'num_layers': 1,'region':7},
    {'seq_length': 8, 'hidden_layer_size': 128, 'num_layers': 2,'region':8},
    {'seq_length': 6, 'hidden_layer_size': 256, 'num_layers': 1,'region':9},
    {'seq_length': 12, 'hidden_layer_size': 128, 'num_layers': 1,'region':10}
    # Add more combinations as needed
]

class RunBuilder():
    @staticmethod
    def get_runs(constant_params, variable_combinations):
        Run = namedtuple('Run', list(constant_params.keys()) + list(variable_combinations[0].keys()))

        runs = []
        for combo in variable_combinations:
            # Merge the constant and variable parameters
            params = {**constant_params, **combo}
            runs.append(Run(**params))

        return runs

# Generate runs with selected combinations
runs = RunBuilder.get_runs(constant_params, variable_combinations)


'''

params = OrderedDict(
    target = ['rate'], ### Our target variable
    lr = [0.00025],
    batch_size = [2],
    seq_length = [4,6,8,12], # length of input sequence to predict values
    region = [1,2,3,4,5,6,7,8,9,10], # specify the region
    output_size = [4], # we are predicting next four weeks admissions
    num_pred_features = [1],
    week_range = [64], # number of historical data (per state) used to train the model
    input_size = [15], # total number of input features
    hidden_layer_size = [64,128,256], #test
    num_layers = [1,2], #test
    ratio = [0.7],  # ratio of training set and validation set
    num_epochs = [50],
    dropout_rate = [0.8],
    lossfunc = [nn.SmoothL1Loss(beta=0.05, reduction = 'sum'),

                ]
)


class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs



runs = RunBuilder.get_runs(params)
'''
pd.options.mode.chained_assignment = None
# output dict
columns = [
    'Model',
    'region',
    'lr',
    'batch_size',
    'seq_length',
    'week_range',
    'hidden_layer_size',
    'num_layers',
    'lossfunc',
    'mse_validation',
    'mse_validation_1w',
    'mse_validation_2w',
    'mse_validation_3w',
    'mse_validation_4w',
    'Target'
]
# output dataframe
df_summary = pd.DataFrame(columns=columns)
for run in RunBuilder.get_runs(constant_params, variable_combinations):
    if run.target == 'rate':
        results_file = 'rate'

    validation_predictions = []
    validation_labels = []
    # read data
    df = pd.read_csv(data_file)
    #df = df.rename(columns={'End Date': 'Week_end', 'State': 'fips'})
    df['Week_end'] = pd.to_datetime(df['Week_end'])
    # too many missing values in State'11'
    df =  df[df['fips'] != 11]
    #df =  df[df['fips'] == 1]
    #df = df[['fips', 'Week_end', 'total_admissions']]

# select region
    df['fips'] = df['fips'].astype(str).str.zfill(2)
    df = df[df['fips'].isin(region_fips[run.region])]
    '''
    # choose data in 2023-11 as test data  test 10-01
    start_date1 = pd.to_datetime('2023-11-01') - timedelta(weeks=run.seq_length )
    df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] < pd.to_datetime('2023-11-18'))] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2023-11 as training data and validation data
    df_train = df[(df['Week_end'] <= pd.to_datetime('2023-11-01'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set
    df_train = df_train.set_index(['fips', 'Week_end'])

    '''
    # test set is the week from 2024-01-06
    start_date1 = pd.to_datetime('2024-01-13') - timedelta(weeks=run.seq_length )
    df_test = df[df['Week_end'] > start_date1] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2024-01-06 as training data and validation data and after 2022-08-01
    df_train = df[(df['Week_end'] <= pd.to_datetime('2024-01-06'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set
    #df_train = df[(df['Week_end'] <= pd.to_datetime('2024-01-06'))& (df['Week_end'] >=pd.to_datetime('2023-01-01')) ]
    df_train = df_train.set_index(['fips', 'Week_end'])

    #rescale the data from 0 to 1, normalization
    first_col = df_train.pop(run.target)
    df_train.insert(0, run.target, first_col)
    first_col_2 = df_test.pop(run.target)
    df_test.insert(0, run.target, first_col_2)

    scaler = MinMaxScaler(feature_range=(0, 1))

    scaler.fit(df_train.iloc[:, 1:])
    train_features_normalized = scaler.transform(df_train.iloc[:, 1:])
    test_feature_normalized = scaler.transform(df_test.iloc[:, 1:])

    scaler_target = MinMaxScaler(feature_range=(0, 1))
    scaler_target.fit(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    train_target_normalized = scaler_target.transform(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    test_target_normalized = scaler_target.transform(np.asarray(df_test.iloc[:, 0]).reshape(-1, 1))

    df_train.iloc[:, 1:] = train_features_normalized
    df_train.iloc[:, 0] = train_target_normalized
    df_test.iloc[:, 1:] = test_feature_normalized
    df_test.iloc[:, 0] = test_target_normalized


    for i in range(1000):

      df_output = pd.DataFrame(columns=['fips', 'Week_end', 'Prediction_1w',
                                        'Prediction_2w', 'Prediction_3w',
                                        'Prediction_4w'])

      test_weeks = df_test[df_test.index.get_level_values('Week_end') \
                        >= pd.to_datetime('2024-01-13')].index.get_level_values('Week_end').unique()
      test_states = df_test.index.get_level_values('fips').unique()

      m_state_dict_main = torch.load(model_file + run.target + '_' + str(run.region) +'_'+
                                    str(run.week_range) + '_' + str(run.hidden_layer_size) \
                                    + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                                    + '_GRU_weights.pt')

      model_main = GRU(run.input_size, run.hidden_layer_size, run.num_layers, run.output_size, run.dropout_rate).to(device)

      model_main.load_state_dict(m_state_dict_main)

      with torch.no_grad():

          for fips in test_states:
              for week in test_weeks:
                  seq = df_test[(df_test.index.get_level_values('fips') == fips)\
                  & (df_test.index.get_level_values('Week_end') <= week)][-run.seq_length:].to_numpy()

                  seq = torch.tensor(seq).reshape(-1, run.seq_length, run.input_size).to(device)

                  model_main.hidden = torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device)
                  prediction = model_main(seq.float())

                  prediction = scaler_target.inverse_transform(prediction.cpu().detach().numpy().reshape(-1, 1))

                  dic = {
                      'fips' : fips,
                      'Week_end' : week,
                      'Prediction_1w' : prediction[0].item(),
                      'Prediction_2w' : prediction[1].item(),
                      'Prediction_3w' : prediction[2].item(),
                      'Prediction_4w' : prediction[3].item()

                  }

                  df_output = pd.concat([df_output, pd.DataFrame([dic])], ignore_index=True)


          df_output.to_csv(prediction_file + run.target + '_' +str(run.region) +'_'+
                              str(run.week_range) + '_' + str(run.hidden_layer_size) \
                              + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                              + '.csv')
          n_result= pd.concat([n_result, df_output], ignore_index=True)


    n_result.to_pickle(summary_file + 'allresulttest0117.pkl')


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
n_result

,fips,Week_end,Prediction_1w,Prediction_2w,Prediction_3w,Prediction_4w
0,09,2023-11-04,0.250918,0.131510,0.281447,0.373191
1,09,2023-11-11,0.211484,0.198454,0.138900,0.260209
2,23,2023-11-04,0.878734,0.782681,1.523751,2.023352
3,23,2023-11-11,0.502271,0.557921,1.701716,1.891219
4,25,2023-11-04,0.693562,1.292118,0.902281,1.169675
...,...,...,...,...,...,...
99995,16,2023-11-11,0.382366,0.444122,0.460472,0.643749
99996,41,2023-11-04,0.153509,0.930195,1.594649,2.156615
99997,41,2023-11-11,0.186033,1.323474,2.378947,3.047188
99998,53,2023-11-04,1.708845,2.491183,2.880956,3.275144
